Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import statistics
from scipy import stats # statistic

Load CSV

In [ ]:
# change to your file location
df_log = pd.read_csv('/content/drive/MyDrive/Škola/DM/[projekt]/logs5.csv', ';')
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/[projekt]/parametricke_odhady.csv', ';')

# empty dict to save created crosstables
dfDict = {}

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
df_log.head

<bound method NDFrame.head of               id      frame    agent  ...  weekCubic weekQuadra crisis
0           2453   883526.0  37885.0  ...          0          0      1
1           2454  1005099.0   4528.0  ...          0          0      1
2           2455   819284.0   3416.0  ...          0          0      1
3           2456  1005099.0   4528.0  ...          0          0      1
4           2457   603493.0     23.0  ...          0          0      1
...          ...        ...      ...  ...        ...        ...    ...
2071230  6405380   968902.0   4275.0  ...     148877    7890481      0
2071231  6405383   673047.0    265.0  ...     148877    7890481      0
2071232  6405393    40118.0     90.0  ...     148877    7890481      0
2071233  6405394    40118.0     90.0  ...     148877    7890481      0
2071234  6405395    40118.0     90.0  ...     148877    7890481      0

[2071235 rows x 24 columns]>

In [ ]:
df.head(5)

,Effect,Intercept,week,week_sq,week_cb,crisis
0,Pricing_List,1.759223,-0.126769,0.005898,-0.000062,-1.407684
1,Reputation,-0.075616,-0.054089,0.003361,-0.000045,-0.577409
2,Business_Conditions,1.154773,-0.220636,0.011659,-0.000136,-2.274406
3,Pillar3_related,0.816321,-0.074923,0.002952,-0.000027,-0.871013
4,Pillar3_disclosure_requirements,-0.004092,0.005430,-0.001337,0.000030,-1.122031


Crosstable for crisis = 0

In [ ]:
df1 = df_log[(df_log['crisis'] == 0)]
crosstable = pd.crosstab(df1['week'], df1['category'], values=df1['crisis'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add crisis for 0 crosstable into dict
dfDict['0'] = crosstable

Crosstable for crisis = 1

In [ ]:
df1 = df_log[(df_log['crisis'] == 1)]
crosstable = pd.crosstab(df1['week'], df1['category'], values=df1['crisis'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing line
crosstable = crosstable.append({'week': 53, 'Business Conditions': 0, 'Pillar3 disclosure requirements': 0, 'Pillar3 related': 0, 'Pricing List': 0, 'Reputation': 0, 'We support..': 0, 'All': 0}, ignore_index=True)

# Add crisis for 1 crosstable into dict
dfDict['1'] = crosstable

Create function for graph estimates creation 

In [ ]:
def create_graph(df_graph, title):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df_graph['0_week'], y=df_graph['1_pricing'],
                         mode='lines',
                         name='Pricing_List'))
  fig.add_trace(go.Scatter(x=df_graph['0_week'], y=df_graph['2_reputation'],
                         mode='lines',
                         name='Reputation'))
  fig.add_trace(go.Scatter(x=df_graph['0_week'], y=df_graph['3_business'],
                         mode='lines',
                         name='Business_Conditions'))
  fig.add_trace(go.Scatter(x=df_graph['0_week'], y=df_graph['4_related'],
                         mode='lines',
                         name='Pillar3_related'))
  fig.add_trace(go.Scatter(x=df_graph['0_week'], y=df_graph['5_disclosure'],
                         mode='lines',
                         name='Pillar3_disclosure_requirements'))
  fig.add_trace(go.Scatter(x=df_graph['0_week'], y=df_graph['6_weSupport'],
                         mode='lines',
                         name='We_support'))
  fig.update_layout(title=title)
  fig.show()

Create array of new category names

In [ ]:
categories = ['1_pricing',	'2_reputation',	'3_business',	'4_related',	'5_disclosure',	'6_weSupport']

Create function to add extreme values

In [ ]:
def add_extreme_values(df1):
  for category in categories:
    df1[category + "_max"] = np.nanmean(df1[category].tolist()) + 2*statistics.stdev(df1[category].tolist())
    df1[category + "_min"] = np.nanmean(df1[category].tolist()) - 2*statistics.stdev(df1[category].tolist())

Create function for graph differences creation

In [ ]:
def create_figure(df1, crisis):
  color_map = {'1_pricing': 'blue', '2_reputation': 'green', '3_business': 'red', '4_related': 'yellow', '5_disclosure': 'violet', '6_weSupport': 'brown'}
  fig = go.Figure()
  for category in categories:
    fig.add_trace(go.Scatter(x=df1['0_week'], y=df1[category], mode='lines', name=category, line=dict(color=color_map[category])))
    fig.add_trace(go.Scatter(x=df1['0_week'], y=df1[category + '_max'], mode='lines', name='ext ' + category, line=dict(color="lightgrey", dash="dashdot")))
    fig.add_trace(go.Scatter(x=df1['0_week'], y=df1[category + '_min'], mode='lines', name='ext ' + category, line=dict(color="lightgrey", dash="dashdot")))

  fig.update_layout(title='Visualization of differences - ' + crisis , xaxis_title='week', yaxis_title='differences')
  return fig

Create function to create figures for logits and return them

In [ ]:
def create_figure_logits(df1, df1_logits, crisis):
  color_map = {'1_pricing': 'blue', '2_reputation': 'green', '3_business': 'red', '4_related': 'yellow', '5_disclosure': 'violet'}
  fig = go.Figure()
  for category in categories[:-1]:
    fig.add_trace(go.Scatter(x=df1['0_week'], y=df1[category], mode='lines', name='teo ' + category, line=dict(color=color_map[category], dash="dashdot")))
    fig.add_trace(go.Scatter(x=df1['0_week'], y=df1_logits[category], mode='lines', name='emp ' + category, line=dict(color=color_map[category])))


  fig.update_layout(title='Visualization of logits differences - ' + crisis, xaxis_title='week', yaxis_title='Logits')
  return fig

## After crisis

In [ ]:
dataframe_collection = {} 

# Create empty dataframe for logits
df_logits = pd.DataFrame()

# Create empty dataframe for estimates
df_estimates = pd.DataFrame()

# Create empty dataframe for evaluation
df_evaluation = pd.DataFrame()

# Create empty dataframe for differences
df_differences = pd.DataFrame()

# Create empty dataframe for empirical relative abundance
df_era = pd.DataFrame()

# Create empty dataframe for empirical logits
df_elogits = pd.DataFrame()

# Cycle through weekdays
for x in range (0, 54):

      # Create logits estimates
      logit_pricing = df.at[0, 'Intercept'] + df.at[0, 'week']*x+df.at[0, 'week_sq']*(x*x)+df.at[0, 'week_cb']*(x*x*x)
      logit_reputation = df.at[1, 'Intercept'] + df.at[1, 'week']*x+df.at[1, 'week_sq']*(x*x)+df.at[1, 'week_cb']*(x*x*x)
      logit_business = df.at[2, 'Intercept'] + df.at[2, 'week']*x+df.at[2, 'week_sq']*(x*x)+df.at[2, 'week_cb']*(x*x*x)
      logit_related = df.at[3, 'Intercept'] + df.at[3, 'week']*x+df.at[3, 'week_sq']*(x*x)+df.at[3, 'week_cb']*(x*x*x)
      logit_disclosure = df.at[4, 'Intercept'] + df.at[4, 'week']*x+df.at[4, 'week_sq']*(x*x)+df.at[4, 'week_cb']*(x*x*x)

      # Reference web
      reference_web = 1 / (1 + np.exp(logit_pricing) + np.exp(logit_reputation) + np.exp(logit_business) + np.exp(logit_related) + np.exp(logit_disclosure))

      # Create estimates
      estimate_pricing = np.exp(logit_pricing) * reference_web
      estimate_reputation = np.exp(logit_reputation) * reference_web
      estimate_business = np.exp(logit_business) * reference_web
      estimate_related = np.exp(logit_related) * reference_web
      estimate_disclosure = np.exp(logit_disclosure) * reference_web

      # Create evaluation
      crosstable = dfDict['0']
      crosstable = crosstable[(crosstable['week'] == x)]
      crosstable_all = crosstable.iloc[0]['All']

      evaluation_pricing = estimate_pricing * crosstable_all
      evaluation_reputation = estimate_reputation * crosstable_all
      evaluation_business = estimate_business * crosstable_all
      evaluation_related = estimate_related * crosstable_all
      evaluation_disclosure = estimate_disclosure * crosstable_all
      evaluation_weSupport = reference_web * crosstable_all

      # Differences
      dij_pricing = crosstable.iloc[0]['Pricing List'] - (estimate_pricing * crosstable_all)
      dij_reputation = crosstable.iloc[0]['Reputation'] - (estimate_reputation * crosstable_all)
      dij_business = crosstable.iloc[0]['Business Conditions'] - (estimate_business * crosstable_all)
      dij_related = crosstable.iloc[0]['Pillar3 related'] - (estimate_related * crosstable_all)
      dij_disclosure = crosstable.iloc[0]['Pillar3 disclosure requirements'] - (estimate_disclosure * crosstable_all)
      dij_weSupport = crosstable.iloc[0]['We support..'] - (reference_web * crosstable_all)

      # Empirical relative abundance
      if(crosstable_all == 0):
        era_pricing = 0
        era_reputation = 0
        era_business = 0
        era_related = 0
        era_disclosure = 0
        era_weSupport = 0
      else:
        era_pricing = crosstable.iloc[0]['Pricing List'] / crosstable_all
        era_reputation = crosstable.iloc[0]['Reputation'] / crosstable_all
        era_business = crosstable.iloc[0]['Business Conditions'] / crosstable_all
        era_related = crosstable.iloc[0]['Pillar3 related'] / crosstable_all
        era_disclosure = crosstable.iloc[0]['Pillar3 disclosure requirements'] / crosstable_all
        era_weSupport = crosstable.iloc[0]['We support..'] / crosstable_all

      # Empirical logits
      if (era_weSupport == 0):
        elogits_pricing = 0
        elogits_reputation = 0
        elogits_business = 0
        elogits_related = 0
        elogits_disclosure = 0
      else:
        elogits_pricing = np.log(era_pricing / era_weSupport)
        elogits_reputation = np.log(era_reputation / era_weSupport)
        elogits_business = np.log(era_business / era_weSupport)
        elogits_related = np.log(era_related / era_weSupport)
        elogits_disclosure = np.log(era_disclosure / era_weSupport)

      # Create new row for logits and append it to dataframe
      new_row_logits = {'0_week': x, '1_pricing': logit_pricing, '2_reputation':logit_reputation, '3_business':logit_business, '4_related':logit_related, '5_disclosure':logit_disclosure}
      df_logits = df_logits.append(new_row_logits, sort=False, ignore_index=True)

      # Create new row for estimates and append it to dataframe
      new_row_estimate = {'0_week': x, '1_pricing': estimate_pricing, '2_reputation':estimate_reputation, '3_business':estimate_business, '4_related':estimate_related, '5_disclosure':estimate_disclosure, '6_weSupport': reference_web}
      df_estimates = df_estimates.append(new_row_estimate, sort=False, ignore_index=True)

      # Create new row for evaluation and append it to dataframe
      new_row_evaluation = {'0_week': x, '1_pricing': evaluation_pricing, '2_reputation':evaluation_reputation, '3_business':evaluation_business, '4_related':evaluation_related, '5_disclosure':evaluation_disclosure, '6_weSupport': evaluation_weSupport}
      df_evaluation = df_evaluation.append(new_row_evaluation, sort=False, ignore_index=True)

      # Create new row for differences and append it to dataframe
      new_row_differences = {'0_week': x, '1_pricing': dij_pricing, '2_reputation':dij_reputation, '3_business':dij_business, '4_related':dij_related, '5_disclosure':dij_disclosure, '6_weSupport': dij_weSupport}
      df_differences = df_differences.append(new_row_differences, sort=False, ignore_index=True)

      # Create new row for empirical relative abundance and append it to dataframe
      new_row_era = {'0_week': x, '1_pricing': era_pricing, '2_reputation':era_reputation, '3_business':era_business, '4_related':era_related, '5_disclosure':era_disclosure, '6_weSupport': era_weSupport}
      df_era = df_era.append(new_row_era, sort=False, ignore_index=True)

      # Create new row for empirical logits and append it to dataframe
      new_row_elogits = {'0_week': x, '1_pricing': elogits_pricing, '2_reputation':elogits_reputation, '3_business':elogits_business, '4_related':elogits_related, '5_disclosure':elogits_disclosure}
      df_elogits = df_elogits.append(new_row_elogits, sort=False, ignore_index=True)

In [ ]:
print("Logits")
df_logits.head(5)

Logits


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure
0,0.0,1.759223,-0.075616,1.154773,0.816321,-0.004092
1,1.0,1.638290,-0.126389,0.945661,0.744322,0.000030
2,2.0,1.528783,-0.170709,0.759051,0.678066,0.001657
3,3.0,1.430333,-0.208842,0.594127,0.617391,0.000968
4,4.0,1.342570,-0.241056,0.450074,0.562137,-0.001858


In [ ]:
print("Estimates")
df_estimates.head(5)

Estimates


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,0.409976,0.065448,0.224001,0.159684,0.070301,0.070589
1,1.0,0.404996,0.069352,0.202603,0.165655,0.078698,0.078696
2,2.0,0.398886,0.072907,0.184739,0.170368,0.086622,0.086478
3,3.0,0.392198,0.076141,0.169960,0.173960,0.093916,0.093825
4,4.0,0.385367,0.079089,0.157859,0.176578,0.100461,0.100648


In [ ]:
print("Evaluation")
df_evaluation.head(5)

Evaluation


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,186.539303,29.778974,101.920430,72.656291,31.986924,32.118077
1,1.0,7910.381126,1354.587020,3957.240193,3235.580420,1537.128958,1537.082284
2,2.0,9534.976647,1742.768837,4416.001496,4072.470988,2070.605229,2067.176804
3,3.0,9092.321747,1765.185469,3940.173979,4032.913030,2177.255983,2175.149792
4,4.0,8959.388250,1838.731246,3670.053677,4105.260234,2335.611281,2339.955312


In [ ]:
print("Differences")
df_differences.head(5)

Differences


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,-60.539303,28.221026,-49.920430,10.343709,-2.986924,74.881923
1,1.0,-341.381126,689.412980,-609.240193,-96.580420,-230.128958,587.917716
2,2.0,-1354.976647,1114.231163,86.998504,-52.470988,-341.605229,547.823196
3,3.0,-1755.321747,1026.814531,-569.173979,470.086970,-150.255983,977.850208
4,4.0,-2100.388250,1096.268754,-354.053677,638.739766,-338.611281,1058.044688


In [ ]:
print("Empirical relative abundance")
df_era.head(5)

Empirical relative abundance


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,0.276923,0.127473,0.114286,0.182418,0.063736,0.235165
1,1.0,0.387518,0.104649,0.171411,0.160711,0.066916,0.108796
2,2.0,0.342202,0.119520,0.188379,0.168173,0.072331,0.109396
3,3.0,0.316482,0.120433,0.145408,0.194237,0.087435,0.136005
4,4.0,0.295023,0.126242,0.142630,0.204052,0.085896,0.146157


In [ ]:
print("Empirical logits")
df_elogits.head(5)

Empirical logits


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure
0,0.0,0.163453,-0.612386,-0.721585,-0.253988,-1.305533
1,1.0,1.270289,-0.038863,0.454591,0.390132,-0.486037
2,2.0,1.140428,0.088508,0.543480,0.430018,-0.413721
3,3.0,0.844576,-0.121596,0.066855,0.356389,-0.441798
4,4.0,0.702375,-0.146480,-0.024428,0.333694,-0.531541


Print WilcoxonResult without Crisis

In [ ]:
for category in categories:
  print(stats.wilcoxon(df_estimates[category], df_era[category]))

WilcoxonResult(statistic=742.0, pvalue=0.9965650306082071)
WilcoxonResult(statistic=650.0, pvalue=0.42577333574335274)
WilcoxonResult(statistic=448.0, pvalue=0.01122198784124478)
WilcoxonResult(statistic=623.0, pvalue=0.3035167755588619)
WilcoxonResult(statistic=722.0, pvalue=0.8598937103385655)
WilcoxonResult(statistic=716.0, pvalue=0.8195134788887095)


In [ ]:
# After crisis
create_graph(df_estimates, 'Estimates after crisis')

Create graph after crisis

In [ ]:
# Add extreme values to each category in dataframe
add_extreme_values(df_differences)
fig = create_figure(df_differences, 'after crisis')
fig.show()

Create graph after crisis for empirical logits

In [ ]:
create_figure_logits(df_logits, df_elogits, 'after crisis')

## During crisis


In [ ]:
dataframe_collection = {} 

# Create empty dataframe for logits and crisis 1
df_logits_crisis = pd.DataFrame()

# Create empty dataframe for estimates and crisis 1
df_estimates_crisis = pd.DataFrame()

# Create empty dataframe for evaluation and crisis 1
df_evaluation_crisis = pd.DataFrame()

# Create empty dataframe for differences and crisis 1
df_differences_crisis = pd.DataFrame()

# Create empty dataframe for empirical relative abundance and crisis 1
df_era_crisis = pd.DataFrame()

# Create empty dataframe for empirical logits
df_elogits_crisis = pd.DataFrame()

# Cycle through weekdays
for x in range (0, 54):

      # Create logits estimates
      logit_pricing_crisis = df.at[0, 'Intercept'] + df.at[0, 'week']*x+df.at[0, 'week_sq']*(x*x)+df.at[0, 'week_cb']*(x*x*x) +df.at[0, 'crisis']
      logit_reputation_crisis = df.at[1, 'Intercept'] + df.at[1, 'week']*x+df.at[1, 'week_sq']*(x*x)+df.at[1, 'week_cb']*(x*x*x) +df.at[1, 'crisis']
      logit_business_crisis = df.at[2, 'Intercept'] + df.at[2, 'week']*x+df.at[2, 'week_sq']*(x*x)+df.at[2, 'week_cb']*(x*x*x) +df.at[2, 'crisis']
      logit_related_crisis = df.at[3, 'Intercept'] + df.at[3, 'week']*x+df.at[3, 'week_sq']*(x*x)+df.at[3, 'week_cb']*(x*x*x) +df.at[3, 'crisis']
      logit_disclosure_crisis = df.at[4, 'Intercept'] + df.at[4, 'week']*x+df.at[4, 'week_sq']*(x*x)+df.at[4, 'week_cb']*(x*x*x) +df.at[4, 'crisis']

      # Reference web
      reference_web_crisis = 1 / (1 + np.exp(logit_pricing_crisis) + np.exp(logit_reputation_crisis) + np.exp(logit_business_crisis) + np.exp(logit_related_crisis) + np.exp(logit_disclosure_crisis))

      # Create estimates
      estimate_pricing_crisis = np.exp(logit_pricing_crisis) * reference_web_crisis
      estimate_reputation_crisis = np.exp(logit_reputation_crisis) * reference_web_crisis
      estimate_business_crisis = np.exp(logit_business_crisis) * reference_web_crisis
      estimate_related_crisis = np.exp(logit_related_crisis) * reference_web_crisis
      estimate_disclosure_crisis = np.exp(logit_disclosure_crisis) * reference_web_crisis

      # Create evaluation
      crosstable = dfDict['1']
      crosstable = crosstable[(crosstable['week'] == x)]
      crosstable_all = crosstable.iloc[0]['All']
      evaluation_pricing_crisis = estimate_pricing_crisis * crosstable_all
      evaluation_reputation_crisis = estimate_reputation_crisis * crosstable_all
      evaluation_business_crisis = estimate_business_crisis * crosstable_all
      evaluation_related_crisis = estimate_related_crisis * crosstable_all
      evaluation_disclosure_crisis = estimate_disclosure_crisis * crosstable_all
      evaluation_weSupport_crisis = reference_web_crisis * crosstable_all

      # Differences
      dij_pricing_crisis = crosstable.iloc[0]['Pricing List'] - (estimate_pricing_crisis * crosstable_all)
      dij_reputation_crisis = crosstable.iloc[0]['Reputation'] - (estimate_reputation_crisis * crosstable_all)
      dij_business_crisis = crosstable.iloc[0]['Business Conditions'] - (estimate_business_crisis * crosstable_all)
      dij_related_crisis = crosstable.iloc[0]['Pillar3 related'] - (estimate_related_crisis * crosstable_all)
      dij_disclosure_crisis = crosstable.iloc[0]['Pillar3 disclosure requirements'] - (estimate_disclosure_crisis * crosstable_all)
      dij_weSupport_crisis = crosstable.iloc[0]['We support..'] - (reference_web_crisis * crosstable_all)

      # Empirical relative abundance
      if(crosstable_all == 0):
        era_pricing_crisis = 0
        era_reputation_crisis = 0
        era_business_crisis = 0
        era_related_crisis = 0
        era_disclosure_crisis = 0
        era_weSupport_crisis = 0
      else:
        era_pricing_crisis = crosstable.iloc[0]['Pricing List'] / crosstable_all
        era_reputation_crisis = crosstable.iloc[0]['Reputation'] / crosstable_all
        era_business_crisis = crosstable.iloc[0]['Business Conditions'] / crosstable_all
        era_related_crisis = crosstable.iloc[0]['Pillar3 related'] / crosstable_all
        era_disclosure_crisis = crosstable.iloc[0]['Pillar3 disclosure requirements'] / crosstable_all
        era_weSupport_crisis = crosstable.iloc[0]['We support..'] / crosstable_all

      # Empirical logits
      if (era_weSupport_crisis == 0):
        elogits_pricing_crisis = 0
        elogits_reputation_crisis = 0
        elogits_business_crisis = 0
        elogits_related_crisis = 0
        elogits_disclosure_crisis = 0
      else:
        elogits_pricing_crisis = np.log(era_pricing_crisis / era_weSupport_crisis)
        elogits_reputation_crisis = np.log(era_reputation_crisis / era_weSupport_crisis)
        elogits_business_crisis = np.log(era_business_crisis / era_weSupport_crisis)
        elogits_related_crisis = np.log(era_related_crisis / era_weSupport_crisis)
        elogits_disclosure_crisis = np.log(era_disclosure_crisis / era_weSupport_crisis)

      # Create new row for logits and append it to dataframe for crisis 1
      new_row_logits_crisis = {'0_week': x, '1_pricing': logit_pricing_crisis, '2_reputation':logit_reputation_crisis, '3_business':logit_business_crisis, '4_related':logit_related_crisis, '5_disclosure':logit_disclosure_crisis, '6_weSupport': reference_web_crisis}
      df_logits_crisis = df_logits_crisis.append(new_row_logits_crisis, sort=False, ignore_index=True)

      # Create new row for estimates and append it to dataframe for crisis 1
      new_row_estimate_crisis = {'0_week': x, '1_pricing': estimate_pricing_crisis, '2_reputation':estimate_reputation_crisis, '3_business':estimate_business_crisis, '4_related':estimate_related_crisis, '5_disclosure':estimate_disclosure_crisis, '6_weSupport': reference_web_crisis}
      df_estimates_crisis = df_estimates_crisis.append(new_row_estimate_crisis, sort=False, ignore_index=True)

      # Create new row for evaluation and append it to dataframe for crisis 1
      new_row_evaluation_crisis = {'0_week': x, '1_pricing': evaluation_pricing_crisis, '2_reputation':evaluation_reputation_crisis, '3_business':evaluation_business_crisis, '4_related':evaluation_related_crisis, '5_disclosure':evaluation_disclosure_crisis, '6_weSupport': evaluation_weSupport_crisis}
      df_evaluation_crisis = df_evaluation_crisis.append(new_row_evaluation_crisis, sort=False, ignore_index=True)

      # Create new row for differences and append it to dataframe for crisis 1
      new_row_differences_crisis = {'0_week': x, '1_pricing': dij_pricing_crisis, '2_reputation':dij_reputation_crisis, '3_business':dij_business_crisis, '4_related':dij_related_crisis, '5_disclosure':dij_disclosure_crisis, '6_weSupport': dij_weSupport_crisis}
      df_differences_crisis = df_differences_crisis.append(new_row_differences_crisis, sort=False, ignore_index=True)

      # Create new row for empirical relative abundance and append it to dataframe for crisis 1
      new_row_era_crisis = {'0_week': x, '1_pricing': era_pricing_crisis, '2_reputation':era_reputation_crisis, '3_business':era_business_crisis, '4_related':era_related_crisis, '5_disclosure':era_disclosure_crisis, '6_weSupport': era_weSupport_crisis}
      df_era_crisis = df_era_crisis.append(new_row_era_crisis, sort=False, ignore_index=True)

       # Create new row for empirical logits and append it to dataframe for crisis 1
      new_row_elogits_crisis = {'0_week': x, '1_pricing': elogits_pricing_crisis, '2_reputation':elogits_reputation_crisis, '3_business':elogits_business_crisis, '4_related':elogits_related_crisis, '5_disclosure':elogits_disclosure_crisis}
      df_elogits_crisis = df_elogits_crisis.append(new_row_elogits_crisis, sort=False, ignore_index=True)

In [ ]:
print("Logits during crisis")
df_logits_crisis.head(5)

Logits during crisis


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,0.351539,-0.653025,-1.119633,-0.054692,-1.126123,0.220304
1,1.0,0.230606,-0.703798,-1.328745,-0.126691,-1.122001,0.236658
2,2.0,0.121099,-0.748118,-1.515355,-0.192947,-1.120374,0.251737
3,3.0,0.022649,-0.786251,-1.680279,-0.253622,-1.121064,0.265485
4,4.0,-0.065114,-0.818465,-1.824332,-0.308876,-1.123890,0.277880


In [ ]:
print("Estimates during crisis")
df_estimates_crisis.head(5)

Estimates during crisis


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,0.313108,0.114661,0.071907,0.208578,0.071442,0.220304
1,1.0,0.298038,0.117075,0.062669,0.208497,0.077062,0.236658
2,2.0,0.284144,0.119136,0.055314,0.207563,0.082106,0.251737
3,3.0,0.271566,0.120941,0.049466,0.206012,0.086530,0.265485
4,4.0,0.260362,0.122575,0.044829,0.204039,0.090315,0.277880


In [ ]:
print("Evaluation during crisis")
df_evaluation_crisis.head(5)

Evaluation during crisis


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,2811.392523,1029.543232,645.653698,1872.826313,641.476554,1978.107680
1,1.0,8221.385045,3229.521694,1728.730337,5751.391348,2125.764052,6528.207524
2,2.0,6769.738774,2838.414254,1317.858405,4945.194951,1956.170736,5997.622881
3,3.0,3951.830544,1759.938849,719.823761,2997.888771,1259.186865,3863.331210
4,4.0,4481.618506,2109.883619,771.642618,3512.128209,1554.584261,4783.142788


In [ ]:
print("Differences during crisis")
df_differences_crisis.head(5)

Differences during crisis


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,-1302.392523,-179.543232,-57.653698,-222.826313,112.523446,1649.892320
1,1.0,-1341.385045,-145.521694,337.269663,631.608652,-155.764052,673.792476
2,2.0,-208.738774,98.585746,590.141595,328.805049,-267.170736,-541.622881
3,3.0,756.169456,-85.938849,305.176239,74.111229,-20.186865,-1029.331210
4,4.0,131.381494,-198.883619,393.357382,-165.128209,193.415739,-354.142788


In [ ]:
print("Empirical relative abundance during crisis")
df_era_crisis.head(5)

Empirical relative abundance during crisis


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure,6_weSupport
0,0.0,0.168059,0.094665,0.065486,0.183762,0.083974,0.404054
1,1.0,0.249411,0.111800,0.074896,0.231394,0.071416,0.261084
2,2.0,0.275383,0.123274,0.080084,0.221364,0.070892,0.229003
3,3.0,0.323529,0.115036,0.070437,0.211105,0.085143,0.194750
4,4.0,0.267995,0.111021,0.067681,0.194446,0.101551,0.257306


In [ ]:
print("Empirical logits during crisis")
df_elogits_crisis.head(5)

Empirical logits during crisis


,0_week,1_pricing,2_reputation,3_business,4_related,5_disclosure
0,0.0,-0.877234,-1.451200,-1.819710,-0.787906,-1.571044
1,1.0,-0.045740,-0.848131,-1.248744,-0.120721,-1.296325
2,2.0,0.184427,-0.619327,-1.050660,-0.033927,-1.172579
3,3.0,0.507574,-0.526473,-1.016997,0.080640,-0.827385
4,4.0,0.040705,-0.840547,-1.335453,-0.280109,-0.929702


Print WilcoxonResult during Crisis

In [ ]:
for category in categories:
  print(stats.wilcoxon(df_estimates_crisis[category], df_era_crisis[category]))

WilcoxonResult(statistic=663.0, pvalue=0.4936517056867479)
WilcoxonResult(statistic=280.0, pvalue=6.827286304734016e-05)
WilcoxonResult(statistic=688.0, pvalue=0.6388860322211583)
WilcoxonResult(statistic=628.0, pvalue=0.32419692937082645)
WilcoxonResult(statistic=576.0, pvalue=0.15168607333163361)
WilcoxonResult(statistic=741.0, pvalue=0.9896953464727177)


In [ ]:
# During crisis
create_graph(df_estimates_crisis, 'Estimates during crisis')

Create graph during crisis

In [ ]:
# Add extreme values to each category in dataframe during crisis
add_extreme_values(df_differences_crisis)
fig = create_figure(df_differences_crisis, 'during crisis')
fig.show()

In [ ]:
create_figure_logits(df_logits_crisis, df_elogits_crisis, 'during crisis')